In [ ]:
%pip install dtw-python
%pip install matplotlib
%pip install datetime
%pip install requests
%pip install numpy

In [ ]:
import numpy as np
import requests as req

In [ ]:
import statistics
from typing import List
import requests as req

class Yahoo:
    def __init__(self):
        pass
    
    
    def get_chart(self,symbol:str)->list:
        headers = {
            'x-api-key': "mTd2MNVwEe3WFgj5PQ66Tbpq0CyeP8Q7ESjpmYZ2"
        }
        try:
            url = f"https://yfapi.net/v8/finance/chart/{symbol}?range=1mo&region=FR&interval=1d&lang=en"
            response = req.request("GET", url, headers=headers)
            return response.json()["chart"]["result"][0]["indicators"]["quote"][0]['close']
        
        except Exception as error:
            return error
        
    def resultats_rapports(self,list_actifs:List[str]) -> dict:
        results = {}
        
        # On construit un dictionnaire avec tous les charts des actifs de la liste
        charts = {actif: self.get_chart(actif) for actif in list_actifs}
        
        
        
        # Pour chaque actif 
        for actif_reference in charts:
            for actif_comparaison in charts:
                results[
                    f"distance rapport premiere valeur {actif_reference} "
                    + actif_comparaison
                ] = self.distance_rapport_premiere_valeur(
                    charts[actif_reference], charts[actif_comparaison]
                )

                results[
                    f"distance rapport moyenne {actif_reference} "
                    + actif_comparaison
                ] = self.distance_rapport_moyenne(
                    charts[actif_reference], charts[actif_comparaison]
                )
        
        return results                


    def distance_rapport_premiere_valeur(self,reference,comparison):  
        newreference = []
        newcomparison = []

        for i in range(len(reference)):
            newreference.append(reference[i] / reference[0])
            newcomparison.append(comparison[i] / comparison[0]) 

        distances = [
            abs(newreference[i] - newcomparison[i])
            for i in range(len(newreference))
        ]

        return {"somme":round(sum(distances),2),
                "moyenne":round(statistics.mean(distances),2)}
    
    def distance_rapport_moyenne(self,reference,comparison):          
        newreference = []
        newcomparison = []

        for i in range(len(reference)):
            newreference.append(reference[i] / statistics.mean(reference))
            newcomparison.append(comparison[i] / statistics.mean(comparison)) 

        distances = [
            abs(newreference[i] - newcomparison[i])
            for i in range(len(newreference))
        ]

        return {"somme":round(sum(distances),2),
                "moyenne":round(statistics.mean(distances),2)}
    
    
    


In [ ]:
## Finding two different dataset

# Getting Apple chart
reference = Yahoo().get_chart("AAPL")
print (reference)

# Getting Microsoft chart
comparison = Yahoo().get_chart("MSFT")
print (comparison)

In [ ]:
distances = []
for i in range (0,len(reference)):
    distance = abs(reference[i]-comparison[i])
    distances.append(distance)

print(distances)

In [ ]:
from pylab import *
for i in range (0,len(reference)):
    x = [i,i]
    y = [reference[i],comparison[i]]
    plot(x, y,"k--")
x = np.linspace(0, len(reference)-1, len(reference))



plot(x, reference, "r", linewidth=4)
plot(x, comparison,"b",linewidth=4)
ylabel("Le prix")

plt.show()

In [ ]:
x = np.linspace(0, len(reference)-1, len(reference))
plot(x,distances)
plt.show()

import statistics
print(sum(distances))
print(statistics.mean(distances))

In [ ]:
import statistics

moyenne = statistics.mean(distances)
print(moyenne)
variationsDistance = []
for i in range (0,len(reference)):
    valeur = distances[i] - moyenne
    variationsDistance.append(valeur)
    
plot(x,variationsDistance)
plt.show()

In [ ]:
resultat = abs(statistics.mean(variationsDistance))
print(resultat)

# On comparer les jeux en ramenant à 0 par rapport à la premiere valeur

In [ ]:
newreference = []
newcomparison = []

for i in range (0,len(reference)):
    newreference.append(reference[i] - reference[0])
    newcomparison.append(comparison[i] - comparison[0]) 


distances = []
for i in range (0,len(newreference)):
    distance = abs(newreference[i]-newcomparison[i])
    distances.append(distance)
    
    
import statistics
print(sum(distances))
print(statistics.mean(distances))

plot(x, newreference, "r", linewidth=4)
plot(x, newcomparison,"b",linewidth=4)
ylabel("Le prix")

plt.show()


x = np.linspace(0, len(reference)-1, len(reference))
plot(x,distances)
plt.show()

# On fait le rapport par rapport à la première valeur (reference)

In [ ]:
newreference = []
newcomparison = []

for i in range (0,len(reference)):
    newreference.append(reference[i] / reference[0])
    newcomparison.append(comparison[i] / comparison[0]) 


distances = []
for i in range (0,len(newreference)):
    distance = abs(newreference[i]-newcomparison[i])
    distances.append(distance)
    
    
import statistics
print(sum(distances))
print(statistics.mean(distances))

plot(x, newreference, "r", linewidth=4)
plot(x, newcomparison,"b",linewidth=4)
ylabel("Le prix")

plt.show()


x = np.linspace(0, len(reference)-1, len(reference))
plot(x,distances)
plt.show()

In [ ]:
assets = ['TSLA','AMZN','MSFT','AAPL']


from pprint import pprint

pprint(Yahoo().resultats_rapports(assets))

In [ ]:
class MBOUM:
    def __init__(self):
        pass
    
    
    def get_chart(self,symbol:str,timeframe=20) -> list:
        headers = {
            'X-Mboum-Secret': "DPEbphxr0mBRzEp956ORDzCn1DFQMX6Xhhf1g5GVw6t6LtJlNmkwmYDGKVed"
        }
        try:
            url = f"https://mboum.com/api/v1/hi/history/?symbol={symbol}&interval=1d&diffandsplits=true"
            response = req.request("GET", url, headers=headers).json()

            """
            result = [
                {
                    "date": datetime.fromtimestamp(int(value)),
                    "value": response[value]["close"],
                }
                for value in response
            ]
            """
            result = [ response['data']['items'][value]["close"] for value in response['data']['items']]

            return result[:timeframe]

        except Exception as error:
            print(response)
            raise error
        
    def resultats_rapports(self,list_actifs:List[str]) -> dict:
        results = {}
        
        # On construit un dictionnaire avec tous les charts des actifs de la liste
        charts = {actif: self.get_chart(actif) for actif in list_actifs}
        
        
        
        # Pour chaque actif 
        for actif_reference in charts:
            for actif_comparaison in charts:
                results[
                    f"distance rapport premiere valeur {actif_reference} "
                    + actif_comparaison
                ] = self.distance_rapport_premiere_valeur(
                    charts[actif_reference], charts[actif_comparaison]
                )

                results[
                    f"distance rapport moyenne {actif_reference} "
                    + actif_comparaison
                ] = self.distance_rapport_moyenne(
                    charts[actif_reference], charts[actif_comparaison]
                )
        
        return results                


    def distance_rapport_premiere_valeur(self,reference,comparison):  
        newreference = []
        newcomparison = []

        for i in range(len(reference)):
            newreference.append(reference[i] / reference[0])
            newcomparison.append(comparison[i] / comparison[0]) 

        distances = [
            abs(newreference[i] - newcomparison[i])
            for i in range(len(newreference))
        ]

        return {"somme":round(sum(distances),2),
                "moyenne":round(statistics.mean(distances),2)}
    
    def distance_rapport_moyenne(self,reference,comparison):          
        newreference = []
        newcomparison = []

        for i in range(len(reference)):
            newreference.append(reference[i] / statistics.mean(reference))
            newcomparison.append(comparison[i] / statistics.mean(comparison)) 

        distances = [
            abs(newreference[i] - newcomparison[i])
            for i in range(len(newreference))
        ]

        return {"somme":round(sum(distances),2),
                "moyenne":round(statistics.mean(distances),2)}

In [ ]:
assets = ['TSLA','AMZN','MSFT','AAPL']


from pprint import pprint

pprint(MBOUM().resultats_rapports(assets))